Importing necessary modules that are required for preprocessing the data

In [0]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

Downloading the stopwords package from nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Generating a set of stopwords of the english language and an object of porter stemmer

In [0]:
stemmer=PorterStemmer() 
stop = set(stopwords.words('english'))

A function to clean the text which takes the input string and in first step it eliminates all the characters which are not alphabets or numerals, in second step words which are less than length 3 are removed, in third step stopwords are removed and in final step stemming is being performed for every word and a combined string is returned by the function.

In [0]:
def cleanText(inputText):
  inputText = re.sub(r'[^A-Za-z0-9\s]+',' ',inputText)
  inputText = re.sub(r'\b\w{1,2}\b',' ',inputText)
  removed =  [i for i in inputText.lower().split() if i not in stop]
  res = " ".join([ stemmer.stem(kw) for kw in removed])
  return res

Some supporting modules have been imported.

In [0]:
import numpy as np
import sklearn 
import math

Modules which are associated with sklearn library are imported including the predefined dataset and classifiers.

In [0]:
from sklearn import metrics
from sklearn import feature_selection
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

Tweets from both the files have been read and their ids and text have been seperated and some preprocessing is done to get it into an appropriate format. After that removal of duplicates is being done and then the tweets are being normalized by the function cleanText() created above.

In [0]:
tweets_15=[]
f=open("/content/drive/My Drive/source_tweets.txt")
tweets_15=f.readlines()
f.close()

tweets_16=[]
f=open("/content/drive/My Drive/source_tweets (1).txt")
tweets_16=f.readlines()
f.close()

tweets_15=[i.replace("\t"," ") for i in tweets_15]
tweets_15=[i.strip("\n") for i in tweets_15]
tweets_15=[[i[:18],i[18:]]for i in tweets_15]

for i in range(len(tweets_15)):
  tweets_15[i][0]=int(tweets_15[i][0])

tweets_16=[i.replace("\t"," ") for i in tweets_16]
tweets_16=[i.strip("\n") for i in tweets_16]
tweets_16=[[i[:18],i[18:]]for i in tweets_16]

for i in range(len(tweets_16)):
  tweets_16[i][0]=int(tweets_16[i][0])

tweets=tweets_15+tweets_16

tweet_id=[]
for i in range(len(tweets)):
  tweet_id.append(tweets[0])

res=[]
for i in range(len(tweet_id)):
  if i not in res:
    res.append(i)

tweet_id=res

normalized_tweets=[]
for i in range(len(tweet_id)):
  normalized_tweets.append([tweets[i][0],cleanText(tweets[i][1])])

Fetching the labels of the tweets as the dataset which is used was generated for veracity classification so for the case of rumours there are 3 labels. But the work in this implementation is only on the first stage of rumour classification system so those labels are converted to a single label rumour while non-rumour is kept as it is.

In [0]:
tweets_15_labels=[]
f=open("/content/drive/My Drive/label.txt")
tweets_15_labels=f.readlines()
f.close()

tweets_15_labels=[i.strip("\n") for i in tweets_15_labels]
tweets_15_labels=[i.replace(":"," ") for i in tweets_15_labels]
tweets_15_labels=[i.split() for i in tweets_15_labels]

for i in range(len(tweets_15_labels)):
  if tweets_15_labels[i][0] in ['unverified','false','true']:
    tweets_15_labels[i][0]="rumour"
  else:
    tweets_15_labels[i][0]=tweets_15_labels[i][0]

tweets_15_labels=[[int(i[1]),i[0]] for i in tweets_15_labels]

tweets_16_labels=[]
f=open("/content/drive/My Drive/label (1).txt")
tweets_16_labels=f.readlines()
f.close()

tweets_16_labels=[i.strip("\n") for i in tweets_16_labels]
tweets_16_labels=[i.replace(":"," ") for i in tweets_16_labels]
tweets_16_labels=[i.split() for i in tweets_16_labels]

for i in range(len(tweets_16_labels)):
  if tweets_16_labels[i][0] in ['unverified','false','true']:
    tweets_16_labels[i][0]="rumour"
  else:
    tweets_16_labels[i][0]=tweets_16_labels[i][0]  

tweets_16_labels=[[int(i[1]),i[0]] for i in tweets_16_labels]

tweet_labels=tweets_15_labels+tweets_16_labels

tweet_lab=[i[1] for i in tweet_labels]

Approach-1:Utilising the predefined dataset and working on the topics of the dataset.

Applying the multinomial naive bayes classifier on the predefined news group dataset. A tuple remove is being used to eliminate the kind of content specified in it from the dataset. After that dataset is being fetched and preprocessing is being done on the dataset.Then utilising the tfidf vectorizer for words and generating numeric vectors for the tweets as well as the dataset and classifying accordingly with the classifier.

After that for each input tweet the max probability of it is being fetched for falling in the particular topic and then as per the threshold value the value is being appended. If the value is greater than the threshold then a lable non-rumour is given as it more likely belongs to the particular class. Accuracy score is being calculated for every threshold value and final average is being given.

In [0]:
classifier = MultinomialNB()

remove = ('headers', 'footers', 'quotes')
data_trained = target_train = target_names = None

data_train = fetch_20newsgroups(subset='train', categories=None,shuffle=True, random_state=42,remove=remove)
 
data_trained = [cleanText(text) for text in data_train.data]
target_train = data_train.target 
target_names = data_train.target_names
   
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data_trained)

classifier.fit(vectors, target_train)

ans=0
l1=[0.99,0.9,0.8,0.7,0.6,0.5]
for j in l1:
  l2=[]
  for i in range(len(normalized_tweets)):
    X_test = cleanText(normalized_tweets[i][1])
    vectors1 = vectorizer.transform([X_test,]) 
    pred = classifier.predict_proba(vectors1)
    if(np.max(pred))>j:
      l2.append("non-rumor")
    else:
      l2.append("rumour")
  score = metrics.accuracy_score(tweet_lab, l2)
  print("Accuracy Score {0}".format(score))
  ans+=score
print("Average Accuracy Score {0}".format(ans/len(l1)))  

Accuracy Score 0.7491334488734835
Accuracy Score 0.7491334488734835
Accuracy Score 0.7491334488734835
Accuracy Score 0.7487001733102253
Accuracy Score 0.7487001733102253
Accuracy Score 0.7474003466204506
Average Accuracy Score 0.7487001733102252


Similar as defined above just utilising the bernoulli naive bayes classifier.

In [0]:
classifier = BernoulliNB()

remove = ('headers', 'footers', 'quotes')
data_trained = target_train = target_names = None

data_train = fetch_20newsgroups(subset='train', categories=None,shuffle=True, random_state=42,remove=remove)
 
data_trained = [cleanText(text) for text in data_train.data]
target_train = data_train.target 
target_names = data_train.target_names
   
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data_trained)

classifier.fit(vectors, target_train)

ans=0
l1=[0.99,0.9,0.8,0.7,0.6,0.5]
for j in l1:
  l2=[]
  for i in range(len(normalized_tweets)):
    X_test = cleanText(normalized_tweets[i][1])
    vectors1 = vectorizer.transform([X_test,]) 
    pred = classifier.predict_proba(vectors1)
    if(np.max(pred))>j:
      l2.append("non-rumor")
    else:
      l2.append("rumour")
  score = metrics.accuracy_score(tweet_lab, l2)
  print("Accuracy Score {0}".format(score))
  ans+=score
print("Average Accuracy Score {0}".format(ans/len(l1)))  

Accuracy Score 0.7361351819757366
Accuracy Score 0.5528596187175043
Accuracy Score 0.45363951473136915
Accuracy Score 0.3799826689774697
Accuracy Score 0.32322357019064124
Accuracy Score 0.28466204506065856
Average Accuracy Score 0.45508376660889666


Approach-2 Generating topics from the dataset and utilising it for further classification instead of predefined topics.

Importing necessary modules 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

A function to clean the text

In [0]:
def convertToken(inputStr):
  inputStr = re.sub(r'[^A-Za-z0-9\s]+',' ',inputStr)
  inputStr = re.sub(r'\b\w{1,2}\b',' ',inputStr)
  removed =  [i for i in inputStr.lower().split() if i not in stop]
  res = " ".join([ kw for kw in removed])
  return res

Function to predict the label as per the topic index

In [0]:
def predictClass(index,topicsTweet):
  rum_count=topicsTweet[index].count('rumour')
  non_count=topicsTweet[index].count('non-rumor')
  if(rum_count>non_count):
    return 'rumour'
  else:
    return 'non-rumour'  

Generating the training and testing data from the dataset on the basis of 70:30 ratio.

In [0]:
t1=[convertToken(i[1]) for i in tweets]

train_data=tweets[:1616]
train_data=[convertToken(i[1]) for i in train_data]

test_data=tweets[1616:]
test_data=[convertToken(i[1]) for i in test_data]

Generating the topics using latent dirichlet allocation and fetching the most important words from the topics.

In [0]:
tf_vectorizer = CountVectorizer(max_features=5000,stop_words='english')
tf = tf_vectorizer.fit_transform(train_data)

lda = LatentDirichletAllocation(n_components=20, max_iter=5,learning_method='online',learning_offset=50.,random_state=0)
lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()

Fetching the most important words of the 20 topics and storing it in a list as well as vectorizing the complete dataset and the topics 

In [0]:
tfidf_vect=TfidfVectorizer()
X=tfidf_vect.fit_transform(t1)

l_topic=[]
for topic_idx, topic in enumerate(lda.components_):
  l_topic.append([tf_feature_names[i] for i in topic.argsort()[:-21:-1]])

l_topic=[" ".join(i) for i in l_topic] 

Y=tfidf_vect.transform(l_topic)

In [15]:
l_topic

['muslim obama shutdown orlando amber alert democrat website shooter syrian registered kfc hand biological adopted jandali abdulfattah father steve jobs',
 'open strategist letter turned defector trump voters beats xojanedotcom realdonaldtrump ausopen andy sam groth australia scores murray british supporters number',
 'weapons chemical isis kurds kobane gold 000 olympics darren wilson team walmart donates using support boycottwalmart win address 378 unauthorized',
 'worker drink bottles murder millions meth layton subway thc serves depression screen task recommends adults doctors aids pepsi soda force',
 'url texas law ebola buy microsoft berniesanders nevada mojang sharia tribunal islamic racism minecraft billion amymek deal magic officially institutional',
 'motorcade vladimir putin posted marc comment leibowitz photo space wizard trump2016 neverhillary endorses grand blacklivesmatter kkk camp hillaryclinton apec alpha',
 'potter harry working alan rickman super book rowling caused b

Forming 20 clusters of the topics which consists of the labels of the tweets which will be used for classification. The clusters are generated by the cosine similarity of the training data and the tweets.

Prediction of the testing data is being done using the cosine similarity and then finding the relevant label from the cluster identified.

In [16]:
topicsTweet1=[]
l_top1=[]
for i in range(20):
  topicsTweet1.append([])
for i in range(len(train_data)):
  max=-10.0
  for j in range(20):
    cos_mat=sklearn.metrics.pairwise.cosine_similarity(X[i],Y[j])
    if(cos_mat>max):
      max=cos_mat
      index=j
  l_top1.append(index)    
  topicsTweet1[index].append(tweet_labels[i][1])  

predAns=[]
for i in range(len(test_data)):
  max=-10.0
  for j in range(20):
    cos_mat=sklearn.metrics.pairwise.cosine_similarity(X[1616+i],Y[j])
    if(cos_mat>max):
      max=cos_mat
      index=j
  predAns.append(predictClass(index,topicsTweet1))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])              

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))        

Accuracy Score 0.7384393063583815


Predicting the probability of the best matched topic and fetching the index using the multinomial naive bayes and then picking relvant class label from the cluster identifed from maximum probability.

In [17]:
classifier = MultinomialNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet1))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.7471098265895953


Predicting the probability of the best matched topic and fetching the index using the bernoulli naive bayes and then picking relvant class label from the cluster identifed from maximum probability.

In [18]:
classifier = BernoulliNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet1))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.7471098265895953


Importing necessary modules for word2vec

In [0]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

Utilising the topics generated above but generating clusters on the basis of word2vec and then classifying them on the basis of cosine similarity among the clusters.

In [20]:
model = Word2Vec(t1,size=300,window=2,sg=0,min_count=1,alpha=0.025)
model.train(t1,total_examples=len(t1),epochs=4)

d_t=[]
for i in t1:
    ans=np.zeros((1,300))
    for j in i:
        ans=np.add(ans,model[j])
    d_t.append(ans)

model.train(l_topic,total_examples=len(l_topic),epochs=4)

dq=[]
for i in l_topic:
    ans=np.zeros((1,300))
    for j in i:
        if j in list(model.wv.vocab):
            ans=np.add(ans,model[j])
    dq.append(ans)

matd=np.zeros((2308,300))
for i in range(len(d_t)):
    matd[i]=np.add(matd[i],d_t[i])

matdq=np.zeros((20,300))
for i in range(len(dq)):
    matdq[i]=np.add(matdq[i],dq[i])    

topicsTweet2=[]
l_top1=[]
for i in range(20):
  topicsTweet2.append([])
cos_mat=sklearn.metrics.pairwise.cosine_similarity(matd,matdq)  
for i in range(len(train_data)):
  max=-10.0
  for j in range(20):
    if(cos_mat[i][j]>max):
      max=cos_mat[i][j]
      index=j
  l_top1.append(index)    
  topicsTweet2[index].append(tweet_labels[i][1])  

predAns=[]
for i in range(len(test_data)):
  max=-10.0
  for j in range(20):
    if(cos_mat[1616+i][j]>max):
      max=cos_mat[1616+i][j]
      index=j
  predAns.append(predictClass(index,topicsTweet2))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])              

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Accuracy Score 0.7442196531791907


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Bernoulli naive bayes for the word2vec model.

In [21]:
classifier = BernoulliNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet2))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.7471098265895953


Multinomial naive bayes for the word2vec model.

In [22]:
classifier = MultinomialNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet2))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.7471098265895953


Generating the topics using non-negative matrix factorization and fetching the most important words from the topics.

In [0]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000,stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(train_data)

nmf = NMF(n_components=20, random_state=1,alpha=.3, l1_ratio=.5).fit(tfidf)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [24]:
tfidf_vect=TfidfVectorizer()
X=tfidf_vect.fit_transform(t1)

l_topic=[]
for topic_idx, topic in enumerate(nmf.components_):
  l_topic.append([tfidf_feature_names[i] for i in topic.argsort()[:-21:-1]])

l_topic=[" ".join(i) for i in l_topic] 

l_topic

['url day zouheir experimenting expect expected expel experience experiences explain expand explorer explosion explosions explosive exporting exposes expansion exists extensions',
 'walker paul fast furious death died url rippaulwalker tragic rodas roger rip crash remembering scrapped delayed car hours star stars',
 'memorial war shot soldier ottawa national parliament canadian url dead cirillo nathan hill ottawashooting uniformed cpl attacker gunman update ottnews',
 'lego parents letter 1970s powerful later message real offers url years sets included gender egalitarian explorer explain experimenting experiences exceed',
 'hands shoot hong kong ferguson protesters gesture using hongkong protests url powerful emulation solidarity image dangillmor missouri edition connection breaking',
 'marlboro related disease smoking dies man ads actor url portrayed played lawson cigarette eric 1970s fox411 forest expansion exhibit exist',
 'mh17 airlines malaysia shot ukraine plane url russian borde

Classifying the topics on the basis of cosine similarity

In [25]:
Y=tfidf_vect.transform(l_topic)

topicsTweet3=[]
l_top1=[]
for i in range(20):
  topicsTweet3.append([])
for i in range(len(train_data)):
  max=-10.0
  for j in range(20):
    cos_mat=sklearn.metrics.pairwise.cosine_similarity(X[i],Y[j])
    if(cos_mat>max):
      max=cos_mat
      index=j
  l_top1.append(index)    
  topicsTweet3[index].append(tweet_labels[i][1])      

predAns=[]
for i in range(len(test_data)):
  max=-10.0
  for j in range(20):
    cos_mat=sklearn.metrics.pairwise.cosine_similarity(X[1616+i],Y[j])
    if(cos_mat>max):
      max=cos_mat
      index=j
  predAns.append(predictClass(index,topicsTweet3))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i]) 

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score)) 

Accuracy Score 0.7471098265895953


Classification using multinomial naive bayes using maximum probability prediction

In [26]:
classifier = MultinomialNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet3))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.7471098265895953


Classification using bernoulli naive bayes using maximum probability prediction

In [27]:
classifier = BernoulliNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet3))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.7471098265895953


Classification on the basis of word2vec model and cosine similarity

In [28]:
model = Word2Vec(t1,size=300,window=2,sg=0,min_count=1,alpha=0.025)
model.train(t1,total_examples=len(t1),epochs=4)
#vectors = model[model.wv.vocab]

d_t=[]
for i in t1:
    ans=np.zeros((1,300))
    for j in i:
        ans=np.add(ans,model[j])
    d_t.append(ans)

model.train(l_topic,total_examples=len(l_topic),epochs=4)
#vectors = model[model.wv.vocab]

dq=[]
for i in l_topic:
    ans=np.zeros((1,300))
    for j in i:
        if j in list(model.wv.vocab):
            ans=np.add(ans,model[j])
    dq.append(ans)

matd=np.zeros((2308,300))
for i in range(len(d_t)):
    matd[i]=np.add(matd[i],d_t[i])

matdq=np.zeros((20,300))
for i in range(len(dq)):
    matdq[i]=np.add(matdq[i],dq[i])    

topicsTweet4=[]
l_top1=[]
for i in range(20):
  topicsTweet4.append([])
cos_mat=sklearn.metrics.pairwise.cosine_similarity(matd,matdq)  
for i in range(len(train_data)):
  max=-10.0
  for j in range(20):
    if(cos_mat[i][j]>max):
      max=cos_mat[i][j]
      index=j
  l_top1.append(index)    
  topicsTweet4[index].append(tweet_labels[i][1])  

predAns=[]
for i in range(len(test_data)):
  max=-10.0
  for j in range(20):
    if(cos_mat[1616+i][j]>max):
      max=cos_mat[1616+i][j]
      index=j
  predAns.append(predictClass(index,topicsTweet4))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])              

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


Accuracy Score 0.7471098265895953


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Classification on word2vec model using bernoulli naive bayes classification with maximum probability

In [29]:
classifier = BernoulliNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet4))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.046242774566473986


Classification on word2vec model using multinomial naive bayes classification with maximum probability

In [30]:
classifier = MultinomialNB()

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_data)

classifier.fit(vectors, l_top1)

predAns=[]
for i in range(len(test_data)):
  vectors1 = vectorizer.transform([test_data[i],]) 
  pred = classifier.predict_proba(vectors1)
  n=pred.argmax()
  predAns.append(predictClass(n,topicsTweet4))
actAns=[]
for i in range(1616,len(tweets)):
  actAns.append(tweet_lab[i])

score = metrics.accuracy_score(actAns, predAns)
print("Accuracy Score {0}".format(score))                

Accuracy Score 0.18352601156069365
